### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [53]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

np.float32(0.07822267)

What's the first value of the resulting vector?

- -0.24
- -0.04
- 0.07
- 0.27

Answer: 0.07

### Q2. Creating the embeddings

In [28]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [29]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [30]:
ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_docs.append(doc)

print(len(ml_docs))

375


In [63]:
import numpy as np

embeddings = []

for d in documents:
    qa_text = '{question} {text}'.format(**d)
    v_qa = embedding_model.encode(qa_text)
    embeddings.append(v_qa)

X = np.array(embeddings)
X.shape

(948, 768)

In [64]:
X

array([[ 0.06168446, -0.0047743 ,  0.06304015, ...,  0.00782937,
        -0.00888188,  0.01962947],
       [ 0.04523693,  0.03428099,  0.04303363, ...,  0.03396188,
        -0.0068176 ,  0.00266172],
       [ 0.06149109, -0.05944145,  0.02415307, ..., -0.00969653,
        -0.01625926,  0.01718603],
       ...,
       [ 0.01600878,  0.00089769,  0.00329854, ..., -0.01636349,
         0.01596112, -0.02984425],
       [-0.05847365, -0.04288519,  0.01265508, ...,  0.02504901,
        -0.00415153, -0.01001441],
       [ 0.0179956 , -0.00797247, -0.02797713, ..., -0.01015308,
         0.00704215,  0.00195756]], dtype=float32)

### Q3. Search

In [65]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

scores = X.dot(v)

In [66]:
v.shape

(768,)

In [67]:
scores.sort()

In [68]:
scores[-1]

np.float32(0.6506573)

### Q4. Hit-rate for our search engine

In [69]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

In [70]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [46]:
idx = 0
doc_id = ground_truth[idx]['document']
question = ground_truth[idx]['question']

print(question)

v_q = embedding_model.encode(question)

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
results = search_engine.search(v_q, num_results=5)

results

Where can I sign up for the course?


[{'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning in a public link you get one point.\n(https://datatalks-club.slack.com/archives/C01FABYF2RG/p1706846846359379?thread_ts=1706825019.546229&cid=C01FABYF2RG)',
  'section': 'General course-related questions',
  'question': 'Homework and Leaderboard - what is the system for points in the course management platform?',
  'course': 'data-engineering-zoomcamp',
  'id': '29865466'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites

In [40]:
doc_id

'0227b872'

In [41]:

relevance = [d['id'] == doc_id for d in results]
relevance

[False, False, False, False, False]

In [71]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [93]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [73]:
def numpy_cosine_search(q):
    question = q['question']

    v_q = embedding_model.encode(question)

    return search_engine.search(v_q, num_results=5)

In [74]:
evaluate(ground_truth, numpy_cosine_search)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9218579234972678, 'mrr': 0.8194262295081971}

## Q5. Indexing with Elasticsearch

In [80]:
len(v)

768

In [88]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [89]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)

  0%|          | 0/948 [00:00<?, ?it/s]

In [90]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [91]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_vector_knn(q):
    question = q
    course = 'machine-learning-zoomcamp'

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

query = 'I just discovered the course. Can I still join it?'

question_vector_knn(query)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

## Q6. Hit-rate for Elasticsearch

In [96]:
def question_vector_knn(q):
    question = q['question']
    course = 'machine-learning-zoomcamp'

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.8076502732240437, 'mrr': 0.6985519125683067}